In [10]:
import torch
import create_cfg_model as cm
import darknet53_model as darknet53

In [2]:
cfg_file = "darknet53.cfg"
weight_file = "darknet53.weights"

blocks = cm.parse_cfg(cfg_file)
# CFG 모델 인스턴스화
cfg_model = cm.Darknet_cfg(blocks)
#  pre-trained weights로 CFG 모델 초기화
cm.load_weights(cfg_model, weight_file)

Finished loading weights


In [3]:
# CFG 모델의 디버깅
cm.debug(cfg_model, input_size=(3, 256, 256))

Input size: (3, 256, 256)
Output size: torch.Size([1, 1000, 8, 8])


In [4]:
model = darknet53.Darknet53()

In [5]:
# 사용이 용이한 모델의 디버깅
darknet53.debug(model, input_size=(3, 256, 256))

Input size: (3, 256, 256)
Output size: torch.Size([1, 1000])


In [8]:
# cfg_model의 가중치를 추출하여 사용이 용이한 모델에 붙여넣기
def transfer_weights(cfg_model, custom_model):
    cfg_params = list(cfg_model.named_parameters())
    custom_params = list(custom_model.named_parameters())

    for (name_cfg, param_cfg), (name_custom, param_custom) in zip(cfg_params, custom_params):
        if param_cfg.data.shape == param_custom.data.shape:
            param_custom.data = param_cfg.data.clone()
            # print(f"{name_cfg}의 레이어 파라미터를 {name_custom}로 전이")
        else:
            print(f"{name_cfg}레이어 파라미터를 {name_custom}에 붙여넣지 못함")

In [9]:
transfer_weights(cfg_model, model)

module_list.76.conv_76.weight레이어 파라미터를 fc.2.weight에 붙여넣지 못함


In [11]:
#전이 완료된 모델 저장하기
MODEL_NAME='DarkNet53'

torch.save(model.state_dict(), f'{MODEL_NAME}.pth')